In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/new_master_dataset.csv")
df

Mounted at /content/drive


,mag,path,filename,class,slide_id,tumor_type
0,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-001.png,benign,22549CD,A
1,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-002.png,benign,22549CD,A
2,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-003.png,benign,22549CD,A
3,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-004.png,benign,22549CD,A
4,100,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_B_A-14-22549CD-100-005.png,benign,22549CD,A
...,...,...,...,...,...,...
7904,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-031.png,malignant,15704,PC
7905,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-032.png,malignant,15704,PC
7906,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-033.png,malignant,15704,PC
7907,400,/content/drive/MyDrive/Breast Cancer Project/b...,SOB_M_PC-14-15704-400-034.png,malignant,15704,PC


In [2]:
groupd_df = df.groupby("tumor_type")

# You can then perform various operations on the grouped data, such as getting the count of each tumor type
tumor_type_counts = groupd_df.size()
tumor_type_counts

tumor_type
A      444
DC    3451
F     1014
LC     626
MC     792
PC     560
PT     453
TA     569
dtype: int64

In [3]:
df_train_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/df_mag100_augmented_final.csv")
df_test_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/test_df_100.csv")
df_val_100 = pd.read_csv("/content/drive/MyDrive/Breast Cancer Project/Mag100/val_df_100.csv")

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Define augmentation parameters
rotation_range = 20
width_shift_range = 0.2
height_shift_range = 0.2
horizontal_flip = True
vertical_flip = True
shear_range = 0.2
zoom_range = 0.2

# Train for Mag 100
train_data = df_train_100
test_data = df_test_100
val_data = df_val_100

# Image preprocessing with augmentation for training
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=rotation_range,
    width_shift_range=width_shift_range,
    height_shift_range=height_shift_range,
    horizontal_flip=horizontal_flip,
    vertical_flip=vertical_flip,
    shear_range=shear_range,
    zoom_range=zoom_range,
    rescale=1./255
)

# Image preprocessing without augmentation for testing and validation
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches using the training generator
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_data,
    x_col='path',
    y_col='tumor_type',
    target_size=(299, 299),
    batch_size=16,
    class_mode='categorical',
    shuffle=True  # Shuffle training data
)

# Flow testing images in batches using the testing generator (without augmentation)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_data,
    x_col='path',
    y_col='tumor_type',
    target_size=(299, 299),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)

# Flow validation images in batches using the validation generator (without augmentation)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_data,
    x_col='path',
    y_col='tumor_type',
    target_size=(299, 299),
    batch_size=4,
    class_mode='categorical',
    shuffle=False
)

Found 4344 validated image filenames belonging to 8 classes.
Found 598 validated image filenames belonging to 8 classes.
Found 397 validated image filenames belonging to 8 classes.


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
import os

In [9]:
# Define the base model (ResNet50) for feature extraction
base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(299, 299, 3))

from tensorflow.keras.models import Model

def resnet50_builder():
    # Specify the layer for feature extraction
    base_in = base_model.input
    base_out = base_model.output

    # Apply Global Average Pooling
    x = GlobalAveragePooling2D()(base_out)

    # Add classification layer
    out = Dense(8, activation='softmax')(x)

    # Define the model
    model = Model(inputs=base_in, outputs=out)

    return model

# Build the ResNet-50 model
model = resnet50_builder()

lr_decay = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=3, verbose=1)
save_model = ModelCheckpoint("/content/drive/MyDrive/resnet50_model_epoch_{epoch:02d}.h5", monitor='loss', period=10, save_best_only=True)

for layer in base_model.layers:
    layer.trainable = False

# Train the classification layers for a few epochs
model.compile(optimizer=Adam(lr=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs=6, validation_data=val_generator, workers=8, callbacks=[lr_decay, save_model])

Epoch 1/3
272/272 [==============================] - 350s 1s/step - loss: 1.8456 - accuracy: 0.3644 - val_loss: 1.8288 - val_accuracy: 0.4207 - lr: 0.0010
Epoch 2/3
272/272 [==============================] - 163s 592ms/step - loss: 1.7998 - accuracy: 0.3794 - val_loss: 1.8871 - val_accuracy: 0.4207 - lr: 0.0010
Epoch 3/3
272/272 [==============================] - 169s 610ms/step - loss: 1.7705 - accuracy: 0.3814 - val_loss: 1.9845 - val_accuracy: 0.2317 - lr: 0.0010


In [ ]:
# Fine-tune the entire model
for layer in model.layers:
    layer.trainable = True

# Compile the model again after fine-tuning
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks to save model at every 10 epochs
checkpoint = ModelCheckpoint("/content/drive/MyDrive/resnet50_model_epochs_{epoch:02d}.h5", monitor='loss', period=10, save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.8, patience=3, verbose=1)

history = model.fit(
    train_generator,
    epochs=70,
    validation_data=val_generator,
    workers=8,
    callbacks=[reduce_lr, checkpoint]
)

Epoch 1/70
272/272 [==============================] - 224s 648ms/step - loss: 1.7027 - accuracy: 0.4664 - val_loss: 10.9152 - val_accuracy: 0.0831 - lr: 0.0010
Epoch 2/70
272/272 [==============================] - 184s 669ms/step - loss: 1.1924 - accuracy: 0.5472 - val_loss: 2.2455 - val_accuracy: 0.1083 - lr: 0.0010
Epoch 3/70
272/272 [==============================] - 181s 658ms/step - loss: 0.9113 - accuracy: 0.6529 - val_loss: 18.6052 - val_accuracy: 0.1083 - lr: 0.0010
Epoch 4/70
272/272 [==============================] - 180s 654ms/step - loss: 0.7587 - accuracy: 0.7143 - val_loss: 4.6532 - val_accuracy: 0.2469 - lr: 0.0010
Epoch 5/70
272/272 [==============================] - 180s 654ms/step - loss: 0.6364 - accuracy: 0.7587 - val_loss: 4.0717 - val_accuracy: 0.4055 - lr: 0.0010
Epoch 6/70
272/272 [==============================] - 179s 652ms/step - loss: 0.5148 - accuracy: 0.8069 - val_loss: 15.5940 - val_accuracy: 0.1058 - lr: 0.0010
Epoch 7/70
272/272 [=======================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


272/272 [==============================] - 183s 667ms/step - loss: 0.3420 - accuracy: 0.8713 - val_loss: 14.9217 - val_accuracy: 0.0932 - lr: 0.0010
Epoch 11/70
272/272 [==============================] - 182s 663ms/step - loss: 0.3155 - accuracy: 0.8893 - val_loss: 6.6697 - val_accuracy: 0.1259 - lr: 0.0010
Epoch 12/70
272/272 [==============================] - 185s 670ms/step - loss: 0.3080 - accuracy: 0.8886 - val_loss: 6.8948 - val_accuracy: 0.1713 - lr: 0.0010
Epoch 13/70
272/272 [==============================] - 182s 664ms/step - loss: 0.2515 - accuracy: 0.9095 - val_loss: 12.5935 - val_accuracy: 0.0529 - lr: 0.0010
Epoch 14/70
272/272 [==============================] - 179s 652ms/step - loss: 0.2323 - accuracy: 0.9151 - val_loss: 8.8542 - val_accuracy: 0.3602 - lr: 0.0010
Epoch 15/70
272/272 [==============================] - 180s 656ms/step - loss: 0.2752 - accuracy: 0.9010 - val_loss: 10.1799 - val_accuracy: 0.1310 - lr: 0.0010
Epoch 16/70
272/272 [============================